# 📊 Day 1 실습 1: 데이터 전처리 및 검증

## 학습 목표
- 한국어 데이터셋을 활용한 RAG 파인튜닝 데이터 준비
- RAFT(Retrieval Augmented Fine Tuning) 방법론 적용
- Instruction/Input/Output 스키마 설계
- 데이터 전처리 규칙 적용 (공백/제어문자/중복/길이/PII 처리)
- 템플릿 고정 및 표준화


In [ ]:
# 필요한 라이브러리 확인 및 설치 (순서대로 진행되는 실습 고려)
import importlib
import subprocess
import sys

def check_and_install_package(package_name, import_name=None, version=None):
    """
    패키지 존재 여부 확인 후 필요시에만 설치
    """
    if import_name is None:
        import_name = package_name.replace('-', '_')
    
    try:
        module = importlib.import_module(import_name)
        print(f"✅ {package_name} 이미 설치됨")
        return True
    except ImportError:
        print(f"📦 {package_name} 설치 중...")
        try:
            if version:
                subprocess.run([sys.executable, "-m", "pip", "install", "-q", f"{package_name}=={version}"], 
                             check=True, capture_output=True)
            else:
                subprocess.run([sys.executable, "-m", "pip", "install", "-q", package_name], 
                             check=True, capture_output=True)
            print(f"✅ {package_name} 설치 완료")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ {package_name} 설치 실패: {e}")
            return False

print("🚀 Day 1 실습 1: 데이터 전처리 및 검증")
print("🔍 필요한 라이브러리 확인 중...")

# 01번 노트북에서 필요한 패키지들
packages = [
    ("datasets", "datasets"),
    ("transformers", "transformers"), 
    ("torch", "torch"),
    ("jsonlines", "jsonlines"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    ("tqdm", "tqdm"),
    ("scikit-learn", "sklearn")
]

print("📋 패키지 확인 결과:")
for package_name, import_name in packages:
    check_and_install_package(package_name, import_name)

print("\n🎉 라이브러리 준비 완료!")
print("💡 다음 셀부터 데이터 전처리를 시작합니다.")

In [ ]:
import json
import random
import re
from typing import Dict, List, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - 그래프에서 한글이 깨지지 않도록 설정
# Colab 환경에서 나눔 글꼴을 설치하고 matplotlib에 적용
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# 폰트 매니저에 폰트 추가 - 그래프에서 한글 표시를 위해 필요
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 모든 그래프에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결
})

print("✅ 한글 폰트 설정 완료 - 그래프에서 한글이 정상 표시됩니다")
print("📦 라이브러리 import 완료!")

## 2. 한국어 데이터셋 로드 및 탐색

### 📋 사용할 데이터셋
- **주요 데이터셋**: `neural-bridge/rag-dataset-12000` (Context-Question-Answer 구조)
- **보조 데이터셋**: `maywell/ko_wikidata_QA` (한국어 QA)
- **RAFT 전처리**: RAG 성능 향상을 위한 context 기반 학습 데이터 생성

In [ ]:
# RAG 데이터셋 로드 (Context-Question-Answer 구조)
print("🔄 RAG 데이터셋 로드 중...")

try:
    # RAG 데이터셋 로드 (이미 context가 있는 구조)
    rag_dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")
    print(f"✅ RAG 데이터셋 로드 완료: {len(rag_dataset)}개 샘플")
    
    # 샘플 데이터 확인
    print("\n📋 데이터 구조:")
    sample = rag_dataset[0]
    print(f"컬럼: {list(sample.keys())}")
    for key, value in sample.items():
        if isinstance(value, str):
            print(f"{key}: {value[:100]}{'...' if len(value) > 100 else ''}")
        else:
            print(f"{key}: {value}")
    
except Exception as e:
    print(f"❌ 데이터셋 로드 실패: {e}")
    print("🔄 대안 데이터셋 사용...")
    
    # 대안: 간단한 한글 RAG 데이터셋 생성
    sample_data = [
        {
            "question": "대한민국의 수도는 어디인가요?",
            "context": "대한민국의 수도는 서울특별시입니다. 서울은 한강을 중심으로 발달한 도시로, 약 950만 명의 인구가 거주하고 있습니다.",
            "answer": "대한민국의 수도는 서울특별시입니다."
        },
        {
            "question": "김치의 주요 재료는 무엇인가요?",
            "context": "김치는 한국의 전통 발효식품입니다. 김치의 주요 재료는 배추, 고춧가루, 마늘, 생강, 젓갈 등으로 구성됩니다. 발효 과정을 통해 유산균이 풍부해집니다.",
            "answer": "김치의 주요 재료는 배추, 고춧가루, 마늘, 생강, 젓갈 등입니다."
        },
        {
            "question": "한국의 전통 음식에는 어떤 것들이 있나요?",
            "context": "한국의 전통 음식으로는 김치, 비빔밥, 불고기, 갈비, 냉면, 삼계탕 등이 있습니다. 이들 음식은 발효, 조림, 구이 등 다양한 조리법을 사용합니다.",
            "answer": "한국의 전통 음식으로는 김치, 비빔밥, 불고기, 갈비, 냉면, 삼계탕 등이 있습니다."
        }
    ]
    rag_dataset = Dataset.from_list(sample_data)
    print(f"✅ 샘플 데이터셋 생성: {len(rag_dataset)}개 샘플")

## 3. 데이터 스키마 정의 및 검증

### 🏗️ Instruction/Input/Output 스키마
- **Instruction**: 모델이 수행해야 할 작업 설명
- **Input**: 작업에 필요한 입력 정보 (질문 + context)
- **Output**: 기대하는 출력 결과 (정답)

In [ ]:
def validate_data_schema(dataset: Dataset) -> Dict[str, Any]:
    """
    데이터셋 스키마 검증 함수
    
    Args:
        dataset: 검증할 데이터셋
        
    Returns:
        검증 결과 딕셔너리
    """
    print("🔍 데이터 스키마 검증 중...")
    
    # 첫 번째 샘플로 컬럼 확인
    if len(dataset) == 0:
        return {"error": "빈 데이터셋"}
    
    sample = dataset[0]
    columns = list(sample.keys())
    total_samples = len(dataset)
    
    # 결측치 및 빈 값 확인
    missing_values = {}
    empty_values = {}
    
    for col in columns:
        missing_count = 0
        empty_count = 0
        
        for item in dataset:
            value = item.get(col)
            if value is None:
                missing_count += 1
            elif isinstance(value, str) and value.strip() == "":
                empty_count += 1
        
        missing_values[col] = missing_count
        empty_values[col] = empty_count
    
    return {
        "total_samples": total_samples,
        "columns": columns,
        "missing_values": missing_values,
        "empty_values": empty_values
    }

# 스키마 검증 실행
validation_result = validate_data_schema(rag_dataset)

# 결과 출력
print(f"\n📊 검증 결과:")
print(f"총 샘플 수: {validation_result['total_samples']}")
print(f"컬럼: {validation_result['columns']}")
print(f"\n결측치 현황:")
for col, count in validation_result['missing_values'].items():
    print(f"  {col}: {count}개")

print(f"\n빈 값 현황:")
for col, count in validation_result['empty_values'].items():
    print(f"  {col}: {count}개")

## 4. 데이터 전처리 규칙 적용

### 🧹 전처리 규칙
1. **공백/제어문자 정리**: 불필요한 공백, 줄바꿈, 특수문자 제거
2. **중복 제거**: 동일한 instruction-input 조합 제거
3. **길이 제한**: 너무 길거나 짧은 샘플 필터링
4. **PII 제거**: 개인정보 패턴 탐지 및 마스킹

In [ ]:
def clean_text(text: str) -> str:
    """
    텍스트 정리 함수: 불필요한 공백, 제어문자 등 제거
    
    Args:
        text: 정리할 텍스트
        
    Returns:
        정리된 텍스트
    """
    if not isinstance(text, str):
        return ""
    
    # 제어문자 제거
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)
    
    # 연속된 공백을 하나로 변환
    text = re.sub(r'\s+', ' ', text)
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text

def remove_pii(text: str) -> str:
    """
    개인정보 제거/마스킹 함수
    
    Args:
        text: PII 제거할 텍스트
        
    Returns:
        PII가 마스킹된 텍스트
    """
    if not isinstance(text, str):
        return ""
    
    # 전화번호 패턴 마스킹
    text = re.sub(r'\d{2,3}-\d{3,4}-\d{4}', '[전화번호]', text)
    text = re.sub(r'\d{3}\d{4}\d{4}', '[전화번호]', text)
    
    # 이메일 패턴 마스킹
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[이메일]', text)
    
    # 주민등록번호 패턴 마스킹 (예: 123456-1234567)
    text = re.sub(r'\d{6}-[1-4]\d{6}', '[주민등록번호]', text)
    
    # 계좌번호 패턴 마스킹 (예: 123-456-789012)
    text = re.sub(r'\d{3}-\d{2,3}-\d{6,8}', '[계좌번호]', text)
    
    return text

def preprocess_rag_dataset(dataset: Dataset, 
                         min_length: int = 10, 
                         max_length: int = 2048) -> Dataset:
    """
    RAG 데이터셋 전처리 함수
    
    Args:
        dataset: 원본 RAG 데이터셋 (context, question, answer 구조)
        min_length: 최소 텍스트 길이
        max_length: 최대 텍스트 길이
        
    Returns:
        전처리된 데이터셋
    """
    print("🧹 RAG 데이터 전처리 시작...")
    
    processed_data = []
    seen_combinations = set()  # 중복 제거용
    
    for item in tqdm(dataset, desc="전처리 중"):
        # 텍스트 정리 (RAG 데이터 구조에 맞춤)
        question = clean_text(item.get('question', ''))
        context = clean_text(item.get('context', ''))
        answer = clean_text(item.get('answer', ''))
        
        # PII 제거
        question = remove_pii(question)
        context = remove_pii(context)
        answer = remove_pii(answer)
        
        # 길이 필터링
        if (len(answer) < min_length or 
            len(context) > max_length or
            len(question) < 5):
            continue
        
        # 중복 확인 (question + context 조합)
        combination = f"{question}|{context}"
        if combination in seen_combinations:
            continue
        seen_combinations.add(combination)
        
        processed_data.append({
            'question': question,
            'context': context,
            'answer': answer
        })
    
    print(f"✅ 전처리 완료: {len(dataset)} → {len(processed_data)}개 샘플")
    return Dataset.from_list(processed_data)

# 전처리 실행
processed_dataset = preprocess_rag_dataset(rag_dataset)
print(f"\n📊 전처리 결과: {len(processed_dataset)}개 샘플")

## 5. RAFT 방법론 적용

### 🎯 RAFT (Retrieval Augmented Fine Tuning) 개념
- **목적**: RAG 시스템의 성능을 향상시키기 위한 특수한 파인튜닝 방법
- **핵심 아이디어**: 모델이 관련 context를 잘 활용하도록 학습
- **방법**: Positive/Negative 샘플을 통한 대조 학습

### 📋 RAFT 데이터 구조
- **Positive 샘플** (60%): 정답 context + 3개 distractor context
- **Negative 샘플** (40%): 4개 distractor context (정답 없음)

In [ ]:
def create_raft_dataset_from_rag(dataset: Dataset, 
                                num_samples: int = 1000,
                                positive_ratio: float = 0.6) -> Dataset:
    """
    RAG 데이터셋으로부터 RAFT 스타일 데이터셋 생성 함수
    
    Args:
        dataset: 원본 RAG 데이터셋 (question, context, answer 구조)
        num_samples: 생성할 총 샘플 수
        positive_ratio: positive 샘플 비율
        
    Returns:
        RAFT 형태로 변환된 데이터셋
    """
    print("🔄 RAFT 데이터셋 생성 중...")
    
    # 샘플 수 제한
    if len(dataset) > num_samples:
        dataset = dataset.shuffle(seed=42).select(range(num_samples))
    
    num_positive = int(len(dataset) * positive_ratio)
    num_negative = len(dataset) - num_positive
    
    # 모든 context를 context pool로 사용
    all_contexts = [item['context'] for item in dataset]
    
    raft_data = []
    used_indices = set()
    
    # Positive 샘플 생성
    print(f"✅ Positive 샘플 생성: {num_positive}개")
    positive_count = 0
    
    for i in tqdm(range(len(dataset)), desc="Positive 샘플 생성"):
        if positive_count >= num_positive:
            break
            
        if i in used_indices:
            continue
            
        item = dataset[i]
        correct_context = item['context']
        question = item['question']
        answer = item['answer']
        
        # 3개의 distractor 선택 (정답이 아닌 다른 context들)
        distractors = random.sample(
            [ctx for j, ctx in enumerate(all_contexts) if j != i and ctx != correct_context], 
            min(3, len(all_contexts) - 1)
        )
        
        # context 리스트 구성 (정답 context를 랜덤 위치에 배치)
        contexts = [correct_context] + distractors
        random.shuffle(contexts)
        
        raft_data.append({
            'type': 'positive',
            'question': question,
            'contexts': contexts,
            'answer': answer,
            'instruction': "주어진 컨텍스트를 바탕으로 질문에 답변해주세요."
        })
        
        used_indices.add(i)
        positive_count += 1
    
    # Negative 샘플 생성
    print(f"✅ Negative 샘플 생성: {num_negative}개")
    negative_count = 0
    
    for i in tqdm(range(len(dataset)), desc="Negative 샘플 생성"):
        if negative_count >= num_negative:
            break
            
        if i in used_indices:
            continue
            
        item = dataset[i]
        question = item['question']
        correct_answer = item['answer']
        correct_context = item['context']
        
        # 4개의 distractor만 선택 (정답 context 제외)
        distractors = random.sample(
            [ctx for j, ctx in enumerate(all_contexts) if j != i and ctx != correct_context],
            min(4, len(all_contexts) - 1)
        )
        
        raft_data.append({
            'type': 'negative', 
            'question': question,
            'contexts': distractors,
            'answer': correct_answer,  # 정답은 있지만 context에는 없음
            'instruction': "주어진 컨텍스트를 바탕으로 질문에 답변해주세요."
        })
        
        used_indices.add(i)
        negative_count += 1
    
    # 데이터 섞기
    random.shuffle(raft_data)
    
    print(f"✅ RAFT 데이터셋 생성 완료: {len(raft_data)}개 샘플")
    print(f"   - Positive: {positive_count}개")
    print(f"   - Negative: {negative_count}개")
    
    return Dataset.from_list(raft_data)

# RAFT 데이터셋 생성
raft_dataset = create_raft_dataset_from_rag(processed_dataset, num_samples=500)

# 샘플 확인
print("\n📋 RAFT 데이터 샘플:")
sample = raft_dataset[0]
print(f"Type: {sample['type']}")
print(f"Question: {sample['question'][:100]}...")
print(f"Contexts: {len(sample['contexts'])}개")
print(f"Answer: {sample['answer'][:100]}...")

## 6. 템플릿 설계 및 고정

### 📝 EXAONE 모델용 Chat Template
- **System Role**: 모델의 역할 정의
- **User Role**: 사용자 입력 (질문 + contexts)
- **Assistant Role**: 모델 응답 (답변)

In [ ]:
def create_chat_template(item: Dict[str, Any]) -> List[Dict[str, str]]:
    """
    EXAONE 모델용 채팅 템플릿 생성
    
    Args:
        item: RAFT 데이터 아이템
        
    Returns:
        채팅 메시지 리스트
    """
    # System 메시지
    system_message = {
        "role": "system",
        "content": (
            "당신은 주어진 컨텍스트를 바탕으로 질문에 정확하고 도움이 되는 답변을 제공하는 AI 어시스턴트입니다. "
            "컨텍스트에서 관련 정보를 찾아 답변하되, 정보가 없다면 모른다고 솔직히 답변하세요."
        )
    }
    
    # User 메시지 - 구조화된 입력
    contexts_text = "\n\n".join([f"컨텍스트 {i+1}: {ctx}" for i, ctx in enumerate(item['contexts'])])
    
    user_content = f"""다음 컨텍스트들을 참고하여 질문에 답변해주세요.

=== 컨텍스트 ===
{contexts_text}

=== 질문 ===
{item['question']}

=== 요청사항 ===
{item['instruction']}"""
    
    user_message = {
        "role": "user", 
        "content": user_content
    }
    
    # Assistant 메시지 (학습시에만 포함)
    assistant_message = {
        "role": "assistant",
        "content": item['answer']
    }
    
    return [system_message, user_message, assistant_message]

def apply_chat_template_to_dataset(dataset: Dataset) -> Dataset:
    """
    데이터셋 전체에 채팅 템플릿 적용
    
    Args:
        dataset: RAFT 데이터셋
        
    Returns:
        템플릿이 적용된 데이터셋
    """
    print("🔄 채팅 템플릿 적용 중...")
    
    templated_data = []
    
    for item in tqdm(dataset, desc="템플릿 적용"):
        chat_messages = create_chat_template(item)
        
        templated_item = {
            'messages': chat_messages,
            'type': item['type'],
            'original_question': item['question'],
            'original_answer': item['answer']
        }
        templated_data.append(templated_item)
    
    print(f"✅ 템플릿 적용 완료: {len(templated_data)}개 샘플")
    return Dataset.from_list(templated_data)

# 템플릿 적용
templated_dataset = apply_chat_template_to_dataset(raft_dataset)

# 템플릿 적용 결과 확인
print("\n📋 템플릿 적용 결과:")
sample = templated_dataset[0]
print(f"Type: {sample['type']}")
print(f"Messages: {len(sample['messages'])}개")
print(f"\n첫 번째 메시지:")
print(f"Role: {sample['messages'][0]['role']}")
print(f"Content: {sample['messages'][0]['content'][:200]}...")

## 7. 토크나이저 로드 및 길이 검증

In [ ]:
# EXAONE 토크나이저 로드
print("🔄 EXAONE 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct")

def analyze_token_lengths(dataset: Dataset, tokenizer, max_length: int = 4096) -> Dict[str, Any]:
    """
    토큰 길이 분석 함수
    
    Args:
        dataset: 분석할 데이터셋
        tokenizer: 사용할 토크나이저
        max_length: 최대 토큰 길이
        
    Returns:
        토큰 길이 분석 결과
    """
    print("📊 토큰 길이 분석 중...")
    
    token_lengths = []
    overflow_count = 0
    
    for item in tqdm(dataset, desc="토큰 길이 계산"):
        # 전체 대화를 하나의 텍스트로 변환
        full_text = tokenizer.apply_chat_template(
            item['messages'], 
            tokenize=False,
            add_generation_prompt=False
        )
        
        # 토큰 개수 계산
        tokens = tokenizer.encode(full_text)
        token_length = len(tokens)
        token_lengths.append(token_length)
        
        if token_length > max_length:
            overflow_count += 1
    
    analysis_result = {
        "total_samples": len(token_lengths),
        "mean_length": np.mean(token_lengths),
        "median_length": np.median(token_lengths),
        "min_length": np.min(token_lengths),
        "max_length": np.max(token_lengths),
        "std_length": np.std(token_lengths),
        "overflow_count": overflow_count,
        "overflow_rate": overflow_count / len(token_lengths),
        "token_lengths": token_lengths
    }
    
    return analysis_result

# 토큰 길이 분석 실행
token_analysis = analyze_token_lengths(templated_dataset, tokenizer)

# 분석 결과 출력
print(f"\n📊 토큰 길이 분석 결과:")
print(f"총 샘플 수: {token_analysis['total_samples']}개")
print(f"평균 길이: {token_analysis['mean_length']:.1f} 토큰")
print(f"중간값: {token_analysis['median_length']:.1f} 토큰")
print(f"최소 길이: {token_analysis['min_length']} 토큰")
print(f"최대 길이: {token_analysis['max_length']} 토큰")
print(f"표준편차: {token_analysis['std_length']:.1f} 토큰")
print(f"4096 토큰 초과: {token_analysis['overflow_count']}개 ({token_analysis['overflow_rate']:.1%})")

## 8. Train/Valid 분할 및 저장

In [ ]:
# Train/Valid 분할 (8:2 비율)
print("🔄 Train/Valid 분할 중...")

train_indices, valid_indices = train_test_split(
    list(range(len(templated_dataset))),
    test_size=0.2,
    random_state=42,
    stratify=[item['type'] for item in templated_dataset]  # positive/negative 비율 유지
)

train_dataset = templated_dataset.select(train_indices)
valid_dataset = templated_dataset.select(valid_indices)

print(f"✅ 분할 완료:")
print(f"  - Train: {len(train_dataset)}개 샘플")
print(f"  - Valid: {len(valid_dataset)}개 샘플")

# 분할 결과 확인
train_types = [item['type'] for item in train_dataset]
valid_types = [item['type'] for item in valid_dataset]

print(f"\n📊 분할 결과 확인:")
print(f"Train - Positive: {train_types.count('positive')}개, Negative: {train_types.count('negative')}개")
print(f"Valid - Positive: {valid_types.count('positive')}개, Negative: {valid_types.count('negative')}개")

In [ ]:
# 저장 디렉토리 생성
import os
os.makedirs('processed_data', exist_ok=True)

# 데이터셋을 JSONL 형식으로 저장
def save_dataset_jsonl(dataset: Dataset, file_path: str):
    """데이터셋을 JSONL 형식으로 저장"""
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# Train/Valid 데이터셋 저장
print("💾 데이터셋 저장 중...")
save_dataset_jsonl(train_dataset, 'processed_data/train_raft_ko.jsonl')
save_dataset_jsonl(valid_dataset, 'processed_data/valid_raft_ko.jsonl')

# 메타데이터 저장
metadata = {
    "dataset_info": {
        "total_samples": len(templated_dataset),
        "train_samples": len(train_dataset),
        "valid_samples": len(valid_dataset),
        "positive_samples": len([item for item in templated_dataset if item['type'] == 'positive']),
        "negative_samples": len([item for item in templated_dataset if item['type'] == 'negative'])
    },
    "token_analysis": {
        "mean_length": float(token_analysis['mean_length']),
        "median_length": float(token_analysis['median_length']),
        "max_length": int(token_analysis['max_length']),
        "overflow_count": int(token_analysis['overflow_count']),
        "overflow_rate": float(token_analysis['overflow_rate'])
    },
    "processing_info": {
        "source_dataset": "neural-bridge/rag-dataset-12000",
        "preprocessing": "RAFT + Chat Template",
        "model_target": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
    }
}

with open('processed_data/metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("✅ 데이터셋 저장 완료:")
print(f"  - processed_data/train_raft_ko.jsonl")
print(f"  - processed_data/valid_raft_ko.jsonl")
print(f"  - processed_data/metadata.json")

## 9. 데이터 품질 검증 및 시각화

In [ ]:
# 토큰 길이 분포 및 데이터 품질 종합 시각화
# 이 차트들은 RAFT 데이터셋의 품질과 특성을 한눈에 파악할 수 있게 해줍니다

plt.figure(figsize=(12, 8))

# 2x2 서브플롯 구성 - 4개의 다른 관점에서 데이터를 분석
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. 토큰 길이 분포 히스토그램 
# 📊 의미: 모델 입력 길이의 분포를 보여줌. 대부분의 데이터가 4096 토큰 이하인지 확인
ax1.hist(token_analysis['token_lengths'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(token_analysis['mean_length'], color='red', linestyle='--', 
           label=f'평균 길이: {token_analysis["mean_length"]:.1f} 토큰')
ax1.axvline(4096, color='orange', linestyle='--', label='모델 최대 길이: 4096 토큰')
ax1.set_xlabel('토큰 길이 (개)')
ax1.set_ylabel('샘플 빈도수')
ax1.set_title('📏 토큰 길이 분포\n(대부분 샘플이 모델 제한 내에 있는지 확인)', fontsize=12, pad=15)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 분포 해석을 위한 추가 정보 표시
ax1.text(0.7, 0.9, f'중간값: {token_analysis["median_length"]:.0f}\n'
                   f'표준편차: {token_analysis["std_length"]:.0f}\n'
                   f'오버플로우: {token_analysis["overflow_count"]}개', 
         transform=ax1.transAxes, fontsize=9, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="wheat", alpha=0.8))

# 2. RAFT Positive/Negative 샘플 비율 파이차트
# 📊 의미: RAFT 방법론의 핵심인 positive/negative 샘플 균형 확인
# - Positive (60%): 정답이 포함된 context로 학습 (정확한 정보 활용 학습)
# - Negative (40%): 정답이 없는 context로 학습 (모른다고 답하는 법 학습)
type_counts = [train_types.count('positive'), train_types.count('negative')]
colors = ['lightcoral', 'lightblue']
wedges, texts, autotexts = ax2.pie(type_counts, labels=['Positive 샘플', 'Negative 샘플'], 
                                  autopct='%1.1f%%', startangle=90, colors=colors)
ax2.set_title('🎯 RAFT 샘플 비율\n(RAG 성능 향상을 위한 대조 학습)', fontsize=12, pad=15)

# 파이차트 텍스트 크기 조정
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 3. 토큰 길이 상세 분포 (박스플롯)
# 📊 의미: 토큰 길이의 사분위수, 이상값 등을 보여줌
# - 중앙값, 1/3분위수로 데이터의 집중도 파악
# - 이상값(outlier)으로 비정상적으로 긴 샘플 식별
box_plot = ax3.boxplot(token_analysis['token_lengths'], patch_artist=True)
box_plot['boxes'][0].set_facecolor('lightgreen')
ax3.set_ylabel('토큰 길이 (개)')
ax3.set_title('📦 토큰 길이 상세 분포\n(사분위수와 이상값 분석)', fontsize=12, pad=15)
ax3.grid(True, alpha=0.3)

# 박스플롯 해석을 위한 주석 추가
q1 = np.percentile(token_analysis['token_lengths'], 25)
q3 = np.percentile(token_analysis['token_lengths'], 75)
ax3.text(1.1, 0.8, f'Q1 (25%): {q1:.0f}\n'
                   f'중앙값: {token_analysis["median_length"]:.0f}\n'
                   f'Q3 (75%): {q3:.0f}', 
         transform=ax3.transAxes, fontsize=9,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.3))

# 4. Context 개수별 분포 막대그래프
# 📊 의미: 각 샘플당 제공되는 context 개수 분포
# - RAFT에서는 보통 4개 context 사용 (1개 정답 + 3개 distractor 또는 4개 distractor)
context_counts = [len(item['contexts']) for item in raft_dataset]
context_count_freq = pd.Series(context_counts).value_counts().sort_index()
bars = ax4.bar(context_count_freq.index, context_count_freq.values, 
               alpha=0.7, color='mediumpurple', edgecolor='black')
ax4.set_xlabel('샘플당 Context 개수')
ax4.set_ylabel('샘플 수')
ax4.set_title('📚 Context 개수별 분포\n(RAFT 구조의 일관성 확인)', fontsize=12, pad=15)
ax4.grid(True, alpha=0.3)

# 막대그래프 위에 값 표시
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height)}개', ha='center', va='bottom', fontweight='bold')

# 전체 레이아웃 조정 및 저장
plt.tight_layout(pad=3.0)  # 서브플롯 간격 조정
plt.savefig('processed_data/data_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 데이터 품질 분석 차트 저장 완료: processed_data/data_analysis.png")
print("\n🔍 차트 해석 가이드:")
print("  📏 토큰 길이 분포: 대부분 샘플이 모델 최대 길이(4096) 이내인지 확인")
print("  🎯 RAFT 샘플 비율: Positive(60%) vs Negative(40%) 균형으로 대조 학습 효과 극대화")
print("  📦 토큰 길이 박스플롯: 데이터 집중도와 이상값으로 품질 문제 파악")  
print("  📚 Context 개수 분포: RAFT 구조의 일관성 확인 (보통 3-4개)")
print("\n💡 이 차트들을 통해 다음 실습에서 사용할 데이터의 품질을 사전 검증했습니다!")

### ✅ 완료된 작업
1. **RAG 데이터셋 로드**: neural-bridge/rag-dataset-12000 활용
2. **데이터 전처리**: 공백/제어문자/중복/길이/PII 처리
3. **RAFT 방법론 적용**: Positive/Negative 샘플 생성
4. **템플릿 표준화**: EXAONE 모델용 Chat Template
5. **토큰 길이 검증**: 4096 토큰 제한 준수 확인
6. **Train/Valid 분할**: 8:2 비율, 균등 분할
7. **데이터 저장**: JSONL 형태로 저장
8. **품질 검증**: 시각화 및 통계 분석

In [ ]:
# 최종 검증 및 요약
print("🎯 Day 1 실습 1 완료 요약")
print("=" * 50)
print(f"📁 저장된 파일:")
print(f"  - processed_data/train_raft_ko.jsonl ({len(train_dataset)}개 샘플)")
print(f"  - processed_data/valid_raft_ko.jsonl ({len(valid_dataset)}개 샘플)")
print(f"  - processed_data/metadata.json")
print(f"  - processed_data/data_analysis.png")

print(f"\n📊 데이터 품질 지표:")
print(f"  - 평균 토큰 길이: {token_analysis['mean_length']:.1f}")
print(f"  - 최대 토큰 길이: {token_analysis['max_length']}")
print(f"  - 4096 토큰 초과율: {token_analysis['overflow_rate']:.1%}")
print(f"  - Positive 샘플: {train_types.count('positive')}개")
print(f"  - Negative 샘플: {train_types.count('negative')}개")

print(f"\n✅ 다음 실습에서 사용할 데이터가 준비되었습니다!")
print(f"   02_data_quality_check.ipynb에서 더 자세한 품질 검증을 진행합니다.")

# 샘플 데이터 미리보기
print(f"\n📋 생성된 데이터 샘플:")
sample_item = train_dataset[0]
print(f"Type: {sample_item['type']}")
print(f"System: {sample_item['messages'][0]['content'][:100]}...")
print(f"User: {sample_item['messages'][1]['content'][:200]}...")
print(f"Assistant: {sample_item['messages'][2]['content'][:100]}...")